# Project 5: Working with Pandas and SQL Databases (Movies Dataset)

# Project Brief for Self-Coders

Here you´ll have the opportunity to code major parts of Project 5 on your own. If you need any help or inspiration, have a look at the Videos or the Jupyter Notebook with the full code. <br> <br>
Keep in mind that it´s all about __getting the right results/conclusions__. It´s not about finding the identical code. Things can be coded in many different ways. Even if you come to the same conclusions, it´s very unlikely that we have the very same code. 

## Creating an SQLite Database

1. __Import__ sqlite3 (as sq3) and __create__ a new SQLite Database with the name __"movies.db"__.

In [37]:
import sqlite3 as sq3
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

In [38]:
con = sq3.connect('movies.db')

In [5]:
con.execute('SELECT * FROM sqlite_master').fetchall()

[]

In [39]:
con.close()

## Loading Data from DataFrames into an SQLite Database

2. __Load__ the json file __"some_movies.json"__ and __split__ the dataset into the following __four datasets__ (save each dataset in a Pandas DataFrame).

__Dataset #1 (Movies)__ with columns ["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]. <br>
Convert "release_date" to datetime and transform "budget" and "revenue" to Million USD before loading into the Database. 

__Dataset #2 (Votes)__ with columns ["id", "vote_count", "vote_average"]. 

__Dataset #3 (Genres)__ with columns ["genre_id", "genre_name", "id"]. <br> 

__Dataset #4 (Prod)__ with columns ["comp_id", "comp_logo_path", "comp_name", "comp_origin_country", "id" ]. <br>


In [110]:
with open('some_movies.json') as openfile:
    data = json.load(openfile)
df = json_normalize(data)

In [111]:
genres = json_normalize(data, record_path = ['genres'], meta = ['id', 'title'], record_prefix = 'genre_')
genres.head(5)

,genre_id,genre_name,id,title
0,12,Adventure,299534,Avengers: Endgame
1,878,Science Fiction,299534,Avengers: Endgame
2,28,Action,299534,Avengers: Endgame
3,28,Action,19995,Avatar
4,12,Adventure,19995,Avatar


In [112]:
prod = json_normalize(data, record_path = ['production_companies'], meta = ['id', 'title'], record_prefix = 'production_')
prod.head()

,production_id,production_logo_path,production_name,production_origin_country,id,title
0,420,/hUzeosd33nzE5MCNsZxCGEKTXaQ.png,Marvel Studios,US,299534,Avengers: Endgame
1,444,/42UPdZl6B2cFXgNUASR8hSt9mpS.png,Dune Entertainment,US,19995,Avatar
2,574,/iB6GjNVHs5hOqcEYt2rcjBqIjki.png,Lightstorm Entertainment,US,19995,Avatar
3,25,/qZCc1lty5FzX30aOCVRBLzaVmcp.png,20th Century Fox,US,19995,Avatar
4,290,/Q8mw2AOQQc8Qg0uNwLWq86DVZv.png,Ingenious Media,GB,19995,Avatar


In [113]:
_ = pd.read_json('some_movies.json', orient = 'records')

In [114]:
df1 = df.loc[:,  ["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]]
df2 = df.loc[:,  ["id", "vote_count", "vote_average"]]
df3 = genres
df4 = prod

In [115]:
df1.revenue = df1.revenue.div(1000000)
df1.budget = df1.budget.div(1000000)

In [147]:
df1.head()

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797.800564,356.0,NaN,2019-04-24
1,19995,Avatar,2787.965087,237.0,NaN,2009-12-10
2,140607,Star Wars: The Force Awakens,2068.223624,245.0,NaN,2015-12-15
3,299536,Avengers: Infinity War,2046.239637,300.0,NaN,2018-04-25
4,597,Titanic,1845.034188,200.0,NaN,1997-11-18


3. __Load__ the datasets __into the database__ (each dataset should be a separate table in the database). __Name__ the tables "Movies", "Votes", "Genres", "Prod".

In [117]:
conn = sq3.connect('movies3.db')

In [118]:
tables = ['Movies', "Votes", "Genres", "Prod"]
db = [df1, df2,df3,df4]          
for a,b in zip(db, tables):
    a.to_sql(b, conn)

## Loading Data from SQLite Databases into DataFrames

4. __Load__ the full tables "Movies", "Votes", "Genres", "Prod" from "movies.db" into Pandas (four DataFrames). __Set__ "id" as Index. 

In [96]:
conn.execute('SELECT * FROM sqlite_master').fetchall()

[('table',
  'Movies',
  'Movies',
  2,
  'CREATE TABLE "Movies" (\n"index" INTEGER,\n  "id" INTEGER,\n  "title" TEXT,\n  "revenue" INTEGER,\n  "budget" INTEGER,\n  "belongs_to_collection_name" REAL,\n  "release_date" TEXT\n)'),
 ('index',
  'ix_Movies_index',
  'Movies',
  3,
  'CREATE INDEX "ix_Movies_index"ON "Movies" ("index")'),
 ('table',
  'Votes',
  'Votes',
  4,
  'CREATE TABLE "Votes" (\n"index" INTEGER,\n  "id" INTEGER,\n  "vote_count" INTEGER,\n  "vote_average" REAL\n)'),
 ('index',
  'ix_Votes_index',
  'Votes',
  5,
  'CREATE INDEX "ix_Votes_index"ON "Votes" ("index")'),
 ('table',
  'Genres',
  'Genres',
  6,
  'CREATE TABLE "Genres" (\n"index" INTEGER,\n  "genre_id" INTEGER,\n  "genre_name" TEXT,\n  "id" INTEGER,\n  "title" TEXT\n)'),
 ('index',
  'ix_Genres_index',
  'Genres',
  7,
  'CREATE INDEX "ix_Genres_index"ON "Genres" ("index")'),
 ('table',
  'Prod',
  'Prod',
  8,
  'CREATE TABLE "Prod" (\n"index" INTEGER,\n  "production_id" INTEGER,\n  "production_logo_path"

In [171]:
movies = pd.read_sql("SELECT * FROM Movies", conn, index_col = 'id', parse_dates = ['release_date'])
votes = pd.read_sql('SELECT * FROM Votes', conn,index_col = 'id')
genres = pd.read_sql("SELECT * FROM Genres", conn, index_col = 'id')
prod = pd.read_sql("SELECT * FROM Prod", conn, index_col = 'id')

In [172]:
prod.head()

,index,production_id,production_logo_path,production_name,production_origin_country,title
id,,,,,,
299534,0,420,/hUzeosd33nzE5MCNsZxCGEKTXaQ.png,Marvel Studios,US,Avengers: Endgame
19995,1,444,/42UPdZl6B2cFXgNUASR8hSt9mpS.png,Dune Entertainment,US,Avatar
19995,2,574,/iB6GjNVHs5hOqcEYt2rcjBqIjki.png,Lightstorm Entertainment,US,Avatar
19995,3,25,/qZCc1lty5FzX30aOCVRBLzaVmcp.png,20th Century Fox,US,Avatar
19995,4,290,/Q8mw2AOQQc8Qg0uNwLWq86DVZv.png,Ingenious Media,GB,Avatar


In [99]:
conn.close()

##  Some Simple SQL Queries

5. __Perform__ the following simple __SQL Queries__ and __store__ the results in DataFrames:

__Load the full "Movies" Table__.

In [120]:
movies = pd.read_sql('SELECT * FROM Movies', conn, index_col = 'id', parse_dates = ['release_date'])
movies.columns

Index(['index', 'title', 'revenue', 'budget', 'belongs_to_collection_name',
       'release_date'],
      dtype='object')

In [121]:
movies.belongs_to_collection_name.tail()

id
330457    None
351286    None
109445    None
321612    None
260513    None
Name: belongs_to_collection_name, dtype: object

__Load the columns "id", "revenue" and "release_date" from "Movies".__ 

In [122]:
movies = pd.read_sql('SELECT id, revenue, release_date FROM Movies', conn)
movies.head()

,id,revenue,release_date
0,299534,2797.800564,2019-04-24
1,19995,2787.965087,2009-12-10
2,140607,2068.223624,2015-12-15
3,299536,2046.239637,2018-04-25
4,597,1845.034188,1997-11-18


__Get the Total Revenue (sum) over all movies from "Movies".__

In [123]:
pd.read_sql('SELECT SUM(revenue) FROM Movies', conn)

,SUM(revenue)
0,29748.575327


__Count the number of Movies in "Movies".__

In [124]:
pd.read_sql('SELECT COUNT(id) FROM Movies', conn)

,COUNT(id)
0,18


__Count the number of Movies that do belong to a collection.__

In [125]:
pd.read_sql('SELECT COUNT(id) FROM Movies WHERE belongs_to_collection_name <> "None"', conn)

,COUNT(id)
0,0


In [127]:
pd.read_sql('SELECT COUNT(belongs_to_collection_name) FROM Movies', conn)

,COUNT(belongs_to_collection_name)
0,0


__Get the average budget from "Movies"__.

In [126]:
pd.read_sql('SELECT AVG(budget) FROM Movies', conn)

,AVG(budget)
0,209.055556


## Some more SQL Queries

6. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Load all columns for the movie with movie id 597__.

In [128]:
pd.read_sql('SELECT * FROM Movies WHERE id == 597', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date
0,4,597,Titanic,1845.034188,200.0,None,1997-11-18


__Load all columns for all movies with a revenue greater than 2000 (MUSD).__

In [131]:
pd.read_sql('SELECT * FROM Movies WHERE revenue > 2000', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date
0,0,299534,Avengers: Endgame,2797.800564,356.0,None,2019-04-24
1,1,19995,Avatar,2787.965087,237.0,None,2009-12-10
2,2,140607,Star Wars: The Force Awakens,2068.223624,245.0,None,2015-12-15
3,3,299536,Avengers: Infinity War,2046.239637,300.0,None,2018-04-25


__Load all columns for all movies with a revenue greater than 1500 (MUSD) and a budget below 200 (MUSD).__

In [133]:
pd.read_sql('SELECT * FROM Movies WHERE revenue > 1500 AND budget < 200', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date
0,5,135397,Jurassic World,1671.713208,150.0,None,2015-06-06
1,8,168259,Furious 7,1515.047671,190.0,None,2015-04-01


__Get the minimum budget from those movies with a revenue greater than 1250 (MUSD).__

In [135]:
pd.read_sql('SELECT MIN(budget) FROM Movies WHERE revenue > 1250', conn)

,MIN(budget)
0,125.0


In [138]:
pd.read_sql('SELECT * FROM Movies WHERE revenue > 1250 AND budget == 125', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date
0,11,12445,Harry Potter and the Deathly Hallows: Part 2,1341.511219,125.0,None,2011-07-07


__Get all unique collection Names from "Movies".__

In [140]:
pd.read_sql('SELECT DISTINCT(title) FROM Movies', conn)

,title
0,Avengers: Endgame
1,Avatar
2,Star Wars: The Force Awakens
3,Avengers: Infinity War
4,Titanic
5,Jurassic World
6,The Lion King
7,The Avengers
8,Furious 7
9,Avengers: Age of Ultron


__Load all movies (all columns) and sort by budget from high to low.__

In [142]:
pd.read_sql('SELECT * FROM Movies ORDER BY budget DESC', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date
0,0,299534,Avengers: Endgame,2797.800564,356.0,None,2019-04-24
1,3,299536,Avengers: Infinity War,2046.239637,300.0,None,2018-04-25
2,6,420818,The Lion King,1656.943394,260.0,None,2019-07-12
3,9,99861,Avengers: Age of Ultron,1405.403694,250.0,None,2015-04-22
4,2,140607,Star Wars: The Force Awakens,2068.223624,245.0,None,2015-12-15
5,1,19995,Avatar,2787.965087,237.0,None,2009-12-10
6,7,24428,The Avengers,1519.557910,220.0,None,2012-04-25
7,4,597,Titanic,1845.034188,200.0,None,1997-11-18
8,10,284054,Black Panther,1346.739107,200.0,None,2018-02-13
9,12,181808,Star Wars: The Last Jedi,1332.539889,200.0,None,2017-12-13


__Load all movies (all columns) that do not belong to a collection.__

In [145]:
pd.read_sql('SELECT * FROM Movies WHERE belongs_to_collection_name IS NULL', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date
0,0,299534,Avengers: Endgame,2797.800564,356.0,None,2019-04-24
1,1,19995,Avatar,2787.965087,237.0,None,2009-12-10
2,2,140607,Star Wars: The Force Awakens,2068.223624,245.0,None,2015-12-15
3,3,299536,Avengers: Infinity War,2046.239637,300.0,None,2018-04-25
4,4,597,Titanic,1845.034188,200.0,None,1997-11-18
5,5,135397,Jurassic World,1671.713208,150.0,None,2015-06-06
6,6,420818,The Lion King,1656.943394,260.0,None,2019-07-12
7,7,24428,The Avengers,1519.557910,220.0,None,2012-04-25
8,8,168259,Furious 7,1515.047671,190.0,None,2015-04-01
9,9,99861,Avengers: Age of Ultron,1405.403694,250.0,None,2015-04-22


__Load all movies (all columns) that belong to a collection.__

In [146]:
pd.read_sql('SELECT * FROM Movies WHERE belongs_to_collection_name IS NOT NULL', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date


__Load all movies (all columns) where "Avengers..." is in the title__.

In [150]:
pd.read_sql('SELECT * FROM Movies WHERE title LIKE "Avengers%"', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date
0,0,299534,Avengers: Endgame,2797.800564,356.0,None,2019-04-24
1,3,299536,Avengers: Infinity War,2046.239637,300.0,None,2018-04-25
2,9,99861,Avengers: Age of Ultron,1405.403694,250.0,None,2015-04-22


## Join Queries

7. __Perform__ the following __SQL Join Queries__ and __store__ the results in DataFrames:

__Join "Movies" and "Votes"__ (all columns).

In [151]:
pd.read_sql('SELECT * FROM Movies INNER JOIN Votes ON Movies.id = Votes.id', conn)

,index,id,title,revenue,budget,belongs_to_collection_name,release_date,index,id,vote_count,vote_average
0,0,299534,Avengers: Endgame,2797.800564,356.0,None,2019-04-24,0,299534,12607,8.3
1,1,19995,Avatar,2787.965087,237.0,None,2009-12-10,1,19995,21000,7.4
2,2,140607,Star Wars: The Force Awakens,2068.223624,245.0,None,2015-12-15,2,140607,14205,7.4
3,3,299536,Avengers: Infinity War,2046.239637,300.0,None,2018-04-25,3,299536,17718,8.3
4,4,597,Titanic,1845.034188,200.0,None,1997-11-18,4,597,16661,7.8
5,5,135397,Jurassic World,1671.713208,150.0,None,2015-06-06,5,135397,15399,6.6
6,6,420818,The Lion King,1656.943394,260.0,None,2019-07-12,6,420818,5425,7.2
7,7,24428,The Avengers,1519.557910,220.0,None,2012-04-25,7,24428,22101,7.7
8,8,168259,Furious 7,1515.047671,190.0,None,2015-04-01,8,168259,7359,7.3
9,9,99861,Avengers: Age of Ultron,1405.403694,250.0,None,2015-04-22,9,99861,15548,7.3


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average").__

In [156]:
pd.read_sql('SELECT Movies.id, Movies.title, Votes.vote_average FROM Votes INNER JOIN Movies ON Votes.id = Movies.id', conn)

,id,title,vote_average
0,299534,Avengers: Endgame,8.3
1,19995,Avatar,7.4
2,140607,Star Wars: The Force Awakens,7.4
3,299536,Avengers: Infinity War,8.3
4,597,Titanic,7.8
5,135397,Jurassic World,6.6
6,420818,The Lion King,7.2
7,24428,The Avengers,7.7
8,168259,Furious 7,7.3
9,99861,Avengers: Age of Ultron,7.3


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8.__

In [167]:
pd.read_sql('SELECT Movies.id, Movies.title, Votes.vote_average \
            FROM Movies \
            JOIN Votes \
            ON Movies.id = Votes.id \
            WHERE Votes.vote_average > 8', conn, index_col = 'id')

,title,vote_average
id,,
299534,Avengers: Endgame,8.3
299536,Avengers: Infinity War,8.3
12445,Harry Potter and the Deathly Hallows: Part 2,8.1


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8 and in ascending budget order__.

In [168]:
pd.read_sql('SELECT Movies.id, Movies.title, Votes.vote_average \
            FROM Movies \
            JOIN Votes \
            ON Movies.id = Votes.id \
            WHERE Votes.vote_average > 8 \
            ORDER BY budget ASC', conn, index_col = 'id')

,title,vote_average
id,,
12445,Harry Potter and the Deathly Hallows: Part 2,8.1
299536,Avengers: Infinity War,8.3
299534,Avengers: Endgame,8.3


## Final Case Study

8. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Get the Total Revenue (sum) for each Production Company.__

In [180]:
pd.read_sql('SELECT Prod.production_name, Movies.revenue, Movies.id \
                 FROM Prod INNER JOIN Movies \
                 ON Movies.id = Prod.id \
                 GROUP BY Prod.production_name \
                 ORDER BY revenue DESC', conn)

,production_name,revenue,id
0,Marvel Studios,2797.800564,299534
1,Lightstorm Entertainment,2787.965087,19995
2,Ingenious Media,2787.965087,19995
3,Dune Entertainment,2787.965087,19995
4,20th Century Fox,2787.965087,19995
5,Truenorth Productions,2068.223624,140607
6,Lucasfilm,2068.223624,140607
7,Bad Robot,2068.223624,140607
8,Paramount,1845.034188,597
9,Universal Pictures,1671.713208,135397


In [190]:
df = pd.read_sql('SELECT Prod.production_name, Movies.revenue, Movies.id, Movies.title \
                 FROM Prod INNER JOIN Movies \
                 ON Movies.id = Prod.id', conn)
df.set_index('id', inplace = True)
df.groupby('production_name').sum().sort_values('revenue', ascending = False)

,revenue
production_name,
Walt Disney Pictures,9446.618940
Marvel Studios,9115.740912
20th Century Fox,4632.999275
Lightstorm Entertainment,4632.999275
Universal Pictures,4490.220464
Lucasfilm,3400.763513
Paramount,3364.592098
Fuji Television Network,3186.760879
Dentsu,3186.760879


__Get all Production Companies for the movie "Titanic".__

In [191]:
pd.read_sql('SELECT Prod.production_name, Movies.title, Movies.id \
             FROM Prod INNER JOIN Movies \
             WHERE Movies.title LIKE "Titanic%"', conn)

,production_name,title,id
0,Marvel Studios,Titanic,597
1,Dune Entertainment,Titanic,597
2,Lightstorm Entertainment,Titanic,597
3,20th Century Fox,Titanic,597
4,Ingenious Media,Titanic,597
5,Truenorth Productions,Titanic,597
6,Lucasfilm,Titanic,597
7,Bad Robot,Titanic,597
8,Marvel Studios,Titanic,597
9,Paramount,Titanic,597


__Get the Total Revenue (sum) for each Genre.__

In [201]:
df = pd.read_sql('SELECT Genres.genre_name, Movies.id, Movies.revenue, Movies.title \
                  FROM Genres INNER JOIN Movies', conn)
df.set_index('id', inplace = True)

df.groupby('genre_name').sum().sort_values('revenue', ascending = False)

,revenue
genre_name,
Adventure,446228.629905
Action,356982.903924
Science Fiction,297485.753270
Family,148742.876635
Fantasy,148742.876635
Animation,89245.725981
Romance,59497.150654
Thriller,59497.150654
Drama,29748.575327


__Get all Genres for the movie "Frozen II".__

In [204]:
pd.read_sql('SELECT Genres.genre_name, Movies.title FROM Genres INNER JOIN Movies ON Movies.id = Genres.id WHERE Movies.title = "Frozen II"', conn)

,genre_name,title
0,Adventure,Frozen II
1,Animation,Frozen II
2,Family,Frozen II


# +++++++++ See some Hints below +++++++++++++

# ++++++++++++++++ Hints++++++++++++++++++++

__Hints for 1.__<br>
You can do this with sq3.connect("database_name.db")

__Hints for 2.__ <br>
You have to use pd.json_normalize(data = ..., record_path = ..., meta = ..., record_prefix = ... ) for Datasets #3 and #4 

__Hints for 3.__<br>
You can do this with: 

In [ ]:
con = sq3.connect("movies.db")
df.to_sql("Table Name", con, index = False)

__Hints for 4.__<br>
You can do this with:

In [ ]:
con = sq3.connect("movies.db")
pd.read_sql("SELECT * FROM Table Name", con, index_col = ...)

__Hints for 5., 6., 7., 8.__<br>
You can do this with:

In [ ]:
con = sq3.connect("movies.db")
df = pd.read_sql("insert the sql query here", con)